# Constant Pressure reactor example
This module supports:
* Concentration distribution
* ROP analysis
* Flux diagram

Function been developing:
* Sensitivity analysis
* Uncertainty analysis 

## 0. Package loading

In [ ]:
using DifferentialEquations
using PythonPlot
using ReactionMechanismSimulator

## 1. Read input file. 
Currently the input file is the superminimal (H2 combustion) example in `/src/testing/`

In [ ]:
#######################    INPUT     #######################
file_path = "/Users/xiaorui/Apps/ReactionMechanismSimulator.jl/src/testing/superminimal.rms";
############################################################

In [ ]:
#load mechanism dictionary
phaseDict = readinput(file_path); 
#mechanism dictionaries index:  phaseDict[phasename]["Species" or "Reactions"]
spcs = phaseDict["phase"]["Species"]; 
rxns = phaseDict["phase"]["Reactions"];

## 2. Parameters
With constant P module, you need to assign two initial parameters out of
* Temperature
* Volume
* Pressure

Besides, you need to assign the initial species distribution in [mol] <br>
If Comparison to Chemkin is wanted, Using V and T instead of P is suggested

In [ ]:
#######################    INPUT     #######################
V0 = 8.314 * 1000 / 1e5;  # [m^3] # Comparison with chemkin
P0 = 1.0e+5; #[Pa]
# T0 = 1000; #[K]
H2 = 0.67;  #[mol]
O2 = 0.33;  #[mol]
N0 = H2 + O2; 
############################################################

In [ ]:
# Define the phase (how species thermodynamic and kinetic properties calculated)
ig = IdealGas(spcs,rxns,name="phase");
# Define initial condition of the reactor
initialconds = Dict(["V"=>V0,"P"=>P0, "H2"=>H2,"O2"=>O2]); 
# Define the domain (encodes how system thermodynamic properties calculated)
domain,y0 = ConstantPDomain(phase=ig,initialconds=initialconds);

## 3. Simulation
In this section, simulation related parameters need to be assigned:
* time span (usually, slightly longer than the length of interets)
* Solver type
* absolute error
* relative error

In [ ]:
#######################    INPUT     #######################
t_final = 2.1; #[s]
solver = DifferentialEquations.CVODE_BDF();
abstol = 1e-20;
reltol = 1e-12;
############################################################

In [ ]:
react = Reactor(domain,y0,(0.0,t_final)); #Create the reactor object

In [ ]:
#solve the ode associated with the reactor
sol = solve(react.ode,solver, abstol=reltol,reltol=reltol);
bsol = Simulation(sol,domain);

## 4. Results
Several results can be displayed:
* Concentration distribution
* Concentration and parameters time profile
* ROP analysis

Built-in functions are available:
*`molefraction()`

but you can also build your customized functions.

In [ ]:
sol(2.0000)[domain.indexes[end]]

In [ ]:
molefractions(bsol,"H2", 1.96)

In [ ]:
getfluxdiagram(bsol,1.96,centralspecieslist=["H2"],radius=5)

In [ ]:
plotmolefractions(bsol, 1.96);

In [ ]:
plotrops(bsol,"H2",1.96)